<a href="https://colab.research.google.com/github/raaraya1/Personal-Proyects/blob/main/Canales%20de%20Youtube/Python%20Engineer/Decision_Tree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Decision Tree**

**Entropy**

$$
E = - \sum p(X) \cdot log_{2}(p(X))
$$

$$
p(X) = \frac{len(x)}{n}
$$

**Ganancia de informacion**

$$
IG = E(parent) - [weight \quad average] \cdot E(children) 
$$

**Metodo (para construir el arbol)**

- Se comienza desde el primer nodo y para cada se selecciona la mejor separacion en base a la ganancia de informacion.
- De la ganancia de informacion mas alta se rescata la variable y el limite.
- Luego se aplica la segmentacion a cada nodo, en base a la variable y limite encontrado.
- Se itera con estos pasos hasta cumplirse algun criterio
 - **maximium depth**: cantidad de nodos maximos al final
 - **minimum samples**: cantidad minima de elementos que puede tener los nodos
 - **no more class distribution**: No existen mas elementos para segmentar

**Aproximacion (prediccion)**

- Se sigue las segmentaciones en el orden del arbol (de arriba a abajo)
- Cuando se llega a un nodo al final del arbol se predice segun el valor mas comun en esa muestra.





## **Armando el algoritmo desde cero**

In [1]:
# Importando bibliotecas
import numpy as np
from collections import Counter

In [2]:
# Funcion de entropia
def entropy(y):
  hist = np.bincount(y)
  ps = hist / len(y)
  return -np.sum([p*np.log2(p) for p in ps if p > 0])

# Se define una clase para cada nodo
## Nota: este almecena la condicion de si corresponde a un nodo final (leaf node)
class Node:
  def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf_node(self):
    return self.value is not None


# Algoritmo de decision tree
class DecisionTree:
  def __init__(self, min_samples_split=2, max_depth=100, n_feats=None):
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_feats = n_feats
    self.root = None

  def fit(self, X, y):
    self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1])
    self.root = self._grow_tree(X, y)

  def _grow_tree(self, X, y, depth=0):
    n_samples, n_features = X.shape 
    n_labels = len(np.unique(y)) # Cantidad de categorias de la variable dependiente

    # Criterios de termino
    if (depth >= self.max_depth
        or n_labels == 1
        or n_samples < self.min_samples_split):
      leaf_value = self._most_common_label(y)
      return Node(value=leaf_value)

    feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

    # Busqueda de la mejor categoria y limite para segmentar
    best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
    left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh)
    left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1)
    right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1)
    return Node(best_feat, best_thresh, left, right)

  # Funcion para obtener la mejor categoria y limite para segmentar
  ## Nota: esta utiliza la funcion de ganancia de informacion en cada categoria y limite, guardando siempre la que obtuvo mejor resultado
  def _best_criteria(self, X, y, feat_idxs):
    best_gain = -1
    split_idx, split_threh = None, None
    for feat_idx in feat_idxs:
      X_column = X[:, feat_idx]
      thresholds = np.unique(X_column)
      for threshold in thresholds:
        gain = self._information_gain(y, X_column, threshold)

        if gain > best_gain:
          best_gain = gain
          split_idx = feat_idx
          split_threh = threshold

    return split_idx, split_threh

  # Funcion para calcular la ganancia en la segmentacion
  def _information_gain(self, y, X_column, split_threh):
    # Entropia del nodo anterior
    parent_entropy = entropy(y)

    # Segmentamos la muestra
    left_idxs, right_idxs = self._split(X_column, split_threh)
    if len(left_idxs) == 0 or len(right_idxs) == 0:
      return 0

    # Ahora aplicamos la funcion de ganancia de informacion restando la entropia
    # ponderada de los nuevos nodos a la entropia del nodo anterior
    n = len(y)
    n_l, n_r = len(left_idxs), len(right_idxs) 
    e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])
    child_entropy = (n_l/n)*e_l + (n_r/n)*e_r    
    ig = parent_entropy - child_entropy
    return ig

  # Funcion para segmetar la muestra en funcion del limite
  def _split(self, X_column, split_threh):
    left_idxs = np.argwhere(X_column <= split_threh).flatten()
    right_idxs = np.argwhere(X_column > split_threh).flatten()
    return left_idxs, right_idxs

  # Funcion para predecir
  # Nota: esta utiliza la funcion para recorrer el arbol
  def predict(self, X):
    return np.array([self._traverse_tree(x, self.root) for x in X])

  # Funcion para recorrer el arbol (recursiva con condicion de termino)
  def _traverse_tree(self, x, node):
    if node.is_leaf_node():
      return node.value

    if x[node.feature] <= node.threshold:
        return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)
 
 # Funcion para seleccionar la categoria mas frecuente en el nodo final
  def _most_common_label(self, y):
    counter = Counter(y)
    most_common = counter.most_common(1)[0][0]
    return most_common


## **Ahora probemos el algoritmo**

In [3]:
# Importamos bibliotecas
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Funcion para medir el acierto del algoritmo
def accuracy(y_true, y_pred):
    accuracy = np.sum(y_true == y_pred) / len(y_true)
    return accuracy

# Cargamos y separamos los datos en los conjuntos de entrenamiento y validacion
data = datasets.load_breast_cancer()
X, y = data.data, data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)

# Llamamos al algoritmo y lo entrenamos
clf = DecisionTree(max_depth=10)
clf.fit(X_train, y_train)

# Generamos las prediccion en base al conjunto de validacion y medimos el acierto que tuvo
y_pred = clf.predict(X_test)
acc = accuracy(y_test, y_pred)
acc

0.9210526315789473

## **Ahora probemos desde la biblioteca de sklearn**

In [4]:
from sklearn.tree import DecisionTreeClassifier

In [5]:
# Cargamos y entremos al clasificador
clf_sk = DecisionTreeClassifier(max_depth=10, random_state=1234)
clf_sk.fit(X_train, y_train)

pred_sk = clf.predict(X_test)
acc_sk = accuracy(y_test, pred_sk)
acc_sk

0.9210526315789473